# Motion generation from checkpoints

In [ ]:
// for local development
%install-location /notebooks/language2motion.gt/swift-install
%install-swiftpm-flags -c release
%install '.package(path: "/notebooks/language2motion.gt")' Datasets TranslationModels TextModels ModelSupport SummaryWriter LangMotionModels Checkpoints

In [ ]:
import TensorFlow
import TextModels
import TranslationModels
import Foundation
import FoundationXML
import ModelSupport
import Datasets
import SummaryWriter
import LangMotionModels
import Checkpoints
import PythonKit

In [ ]:
let np  = Python.import("numpy")

In [ ]:
%include "EnableIPythonDisplay.swift"
IPythonDisplay.shell.enable_matplotlib("inline")

## Set training params

In [ ]:
let device = Device.defaultTFEager

In [ ]:
let maxTextSequenceLength =  20
let maxMotionLength =  100

In [ ]:
let datasetSize: DatasetSize = .full
let batchSize = 150

In [ ]:
let dataURL = URL(fileURLWithPath: "/notebooks/language2motion.gt/data/")
let motionDatasetURL = dataURL.appendingPathComponent("motion_dataset_v3.10Hz.\(datasetSize.rawValue)plist")
// let langDatasetURL = dataURL.appendingPathComponent("labels_ds_v2.csv")

In [ ]:
/// instantiate text processor
let vocabularyURL = dataURL.appendingPathComponent("vocab.txt")
let vocabulary: Vocabulary = try! Vocabulary(fromFile: vocabularyURL)
let tokenizer: Tokenizer = BERTTokenizer(vocabulary: vocabulary, caseSensitive: false, unknownToken: "[UNK]", maxTokenLength: nil)
let textProcessor = TextProcessor(vocabulary: vocabulary, tokenizer: tokenizer)

/// instantiate model
let vocabSize = vocabulary.count
let nbJoints = 47 // TODO: get value from dataset
let nbMixtures = 20
let layerCount: Int = 6
let modelSize: Int = 256
let feedForwardSize: Int = 1024
let headCount: Int = 8
let dropoutProbability: Double = 0.1

In [ ]:
/// load dataset
print("\nLoading dataset...")

var dataset = try Lang2Motion(
    motionDatasetURL: motionDatasetURL,
    batchSize: batchSize,
    trainTestSplit: 1.0,
    device: device
) { (motionSample: MotionSample) -> LangMotionBatch in    
    let sentence = textProcessor.preprocess(sentence: motionSample.annotations[0], maxTextSequenceLength: maxTextSequenceLength)
    let (target2, motionPart) = LangMotionBatch.preprocessTargetMotion(sampleID: motionSample.sampleID, motion: motionSample.motion, maxMotionLength: maxMotionLength)
    let source = LangMotionBatch.Source(sentence: sentence, motionPart: motionPart)
    let singleBatch = LangMotionBatch(data: source,label: target2)
    return singleBatch
}

print("Dataset acquired.")

## helpers

In [ ]:
// let np  = Python.import("numpy")

public func randomNumber(probabilities: [Double]) -> Int {
    // https://stackoverflow.com/questions/30309556/generate-random-numbers-with-a-given-distribution
    // Sum of all probabilities (so that we don't have to require that the sum is 1.0):
    let sum = probabilities.reduce(0, +)
    // Random number in the range 0.0 <= rnd < sum :
    let rnd = Double.random(in: 0.0 ..< sum)
    // Find the first interval of accumulated probabilities into which `rnd` falls:
    var accum = 0.0
    for (i, p) in probabilities.enumerated() {
        accum += p
        if rnd < accum {
            return i
        }
    }
    // This point might be reached due to floating point inaccuracies:
    return (probabilities.count - 1)
}

public func gaussian_pdf(sample: Tensor<Float>, means: Tensor<Float>, variances: Tensor<Float>) -> Tensor<Float> {
    // one-dim tensors
    assert(sample.shape.count == 1)
    assert(sample.shape == means.shape)
    assert(sample.shape == variances.shape)
    let a1 = sqrt(Float(2.0) * Float(np.pi)! * variances)
    let a2 = -(sample - means).squared()
    return Float(1.0) / a1 * exp(a2 / (2.0 * variances))
}

public func bernoulli_pdf(sample: Int, p: Float) -> Float {
    let fSample = Float(sample)
    return fSample * p + Float(1.0 - fSample) * (1.0 - p)
}

public class MotionDecoder2 {
    public static func performNormalMixtureSampling(preds: MixtureModelPreds, nb_joints: Int, nb_mixtures: Int, maxMotionLength: Int) -> (motion: Tensor<Float>, log_probs: [Float], done: Tensor<Int32>) {
        let TINY: Float = 1e-8
        let motionLength = preds.mixtureMeans.shape[1]

        var motion: Tensor<Float> = Tensor<Float>(zeros: [motionLength, nb_joints])
        var log_probs: [Float] = [Float](repeating:0.0, count: motionLength)
        var done: [Int32] = [Int32](repeating: 0, count: motionLength)

        let all_means = preds.mixtureMeans.squeezingShape(at: 0)
        let all_variances = preds.mixtureVars.squeezingShape(at: 0) + TINY
        let weights = preds.mixtureWeights.squeezingShape(at: 0)
        let stops = preds.stops[0, 0..., 0]

        /// Sample joint values.
        var samples = Tensor<Float>(zeros: [motionLength, nb_joints])
        var means = Tensor<Float>(zeros: [motionLength, nb_joints])
        var variances = Tensor<Float>(zeros: [motionLength, nb_joints])
        for width_idx in 0..<motionLength {
            // Decide which mixture to sample from
            let p = weights[width_idx].scalars.map { Double($0)}
            assert(p.count == nb_mixtures)
            let mixture_idx = randomNumber(probabilities: p) //np.random.choice(range(nb_mixtures), p=p)

            /// Sample from it.
            let start_idx = mixture_idx * nb_joints
            let m = all_means[width_idx, start_idx..<start_idx + nb_joints]
            let v = all_variances[width_idx, start_idx..<start_idx + nb_joints]
            assert(m.shape == [nb_joints])
            assert(m.shape == v.shape)
            // https://numpy.org/doc/stable/reference/random/generated/numpy.random.normal.html
            let s = np.random.normal(m.scalars, v.scalars)
            samples[width_idx, 0...] = Tensor<Float>(Array(s)!)
            means[width_idx, 0...] = m
            variances[width_idx, 0...] = v
        }

        for idx in 0..<motionLength {
            let sample = samples[idx]
            let stop: Float = stops[idx].scalar!
            // if done[idx] != 0 {
            //     continue
            // }
            motion[idx] = sample
            // https://docs.scipy.org/doc/numpy-1.14.0/reference/generated/numpy.random.binomial.html
            let sampled_stop: Int = Int(np.random.binomial(n: 1, p: stop))!
            log_probs[idx] += log(gaussian_pdf(sample: sample, means: means[idx], variances: variances[idx])).sum().scalar!
            log_probs[idx] += log(bernoulli_pdf(sample: sampled_stop, p: stop))
            done[idx] = (sampled_stop == 0) ? 1 : 0
        }
        return (motion: motion, log_probs: log_probs, done: Tensor(done))
    }
    
    public static func greedyDecodeMotion(
        sentence: LangMotionBatch.Sentence, 
        transformer: LangMotionTransformer, 
        nbJoints: Int, 
        nbMixtures: Int, 
        maxMotionLength: Int,
        memoryMultiplier: Float = 1.0
    ) -> Tensor<Float> {
        print("\nEncode:")
        print("======")
        let memory = transformer.encode(input: sentence) * memoryMultiplier
        print("  memory.count: \(memory.shape)")     

        print("\nGenerate:")
        print("=========")
        // start with tensor for neutral motion frame
        var ys: Tensor<Float> = Tensor<Float>(repeating:0.0, shape: [1, 1, nbJoints])
        for _ in 0..<maxMotionLength {
            // prepare input
            let motionPartMask = Tensor<Float>(subsequentMask3(size: ys.shape[1]))
            let motionPart = LangMotionBatch.MotionPart(motion: ys, mask: motionPartMask)

            // decode motion
            let decoded = transformer.decode(sourceMask: sentence.mask, motionPart: motionPart, memory: memory)            
            let singlePreds = transformer.mixtureModel(decoded[0...,-1].expandingShape(at: 0))
            
            // perform sampling
            let (sampledMotion, _, _) = MotionDecoder.performNormalMixtureSampling(
                preds: singlePreds, nb_joints: nbJoints, nb_mixtures: nbMixtures, maxMotionLength: maxMotionLength)
            
            // concatenate motion
            ys = Tensor(concatenating: [ys, sampledMotion.expandingShape(at: 0)], alongAxis: 1)        
        }
        return ys.squeezingShape(at:0)
    }    
}

In [ ]:
public func greedyDecodeMotion(model: LangMotionTransformer, sentence: String, prefix: String = "prefix", saveMotion: Bool = true, memoryMultiplier: Float = 0.0, motionsURL: URL?) {
    // TODO: incorporate done/stop signal
    Context.local.learningPhase = .inference
    print("\ngreedyDecodeMotion(sentence: \"\(sentence)\")")

    let processedSentence = textProcessor.preprocess(sentence: sentence, maxTextSequenceLength: maxTextSequenceLength)
    processedSentence.printSentence()

    let decodedMotion = MotionDecoder2.greedyDecodeMotion(
        sentence: processedSentence, 
        transformer: model, 
        nbJoints: nbJoints, 
        nbMixtures: nbMixtures, 
        maxMotionLength: maxMotionLength,
        memoryMultiplier: memoryMultiplier
    )
    print("  decodedMotion: min: \(decodedMotion.min()), max: \(decodedMotion.max())")
    let descaledMotion = dataset.scaler.inverse_transform(decodedMotion)
    print("  descaledMotion.shape: \(descaledMotion.shape)")
    print("  descaledMotion: min: \(descaledMotion.min()), max: \(descaledMotion.max())")
    var imageURL: URL? = nil
    
    if !saveMotion { imageURL = nil } else {
        imageURL = motionsURL!.appendingPathComponent("\(prefix).png")
    }
    motionToImg(url: imageURL, motion: descaledMotion, motionFlag: nil, padTo: maxMotionLength, descr: "\(prefix), \(sentence)", cmapRange: 2.0)

    if saveMotion {
        print("Saved image: \(imageURL!.path)")
        let jointNames = dataset.motionSamples[0].jointNames
        let mmmXMLDoc = MMMWriter.getMMMXMLDoc(jointNames: jointNames, motion: descaledMotion)
        let mmmURL = motionsURL!.appendingPathComponent("\(prefix).mmm.xml")
        try! mmmXMLDoc.xmlData(options: XMLNode.Options.nodePrettyPrint).write(to: mmmURL)
        print("Saved motion: \(mmmURL.path)")
    }
}

## load model checkpoint

In [ ]:
// config
let config = LangMotionTransformerConfig(
    vocabSize: vocabSize,
    nbJoints: nbJoints,
    nbMixtures: nbMixtures,
    layerCount: layerCount,
    modelSize: modelSize,
    feedForwardSize: feedForwardSize,
    headCount: headCount,
    dropoutProbability: dropoutProbability
)

In [ ]:
let runName = "run_7"

In [ ]:
let runURL = dataURL.appendingPathComponent("runs/Lang2motion/\(runName)", isDirectory: true)
let checkpointURL = runURL.appendingPathComponent("checkpoints", isDirectory: true)
let motionsURL = runURL.appendingPathComponent("generated_motions", isDirectory: true)
try! FileManager().createDirectory(at: motionsURL, withIntermediateDirectories: true)

In [ ]:
let epoch = 1

In [ ]:
let model = LangMotionTransformer(checkpoint: checkpointURL, config: config, name: "model.e\(epoch)")
// let model = LangMotionTransformer(checkpoint: checkpointURL, config: config, name: "model.final")

## decode motion

In [ ]:
greedyDecodeMotion(
    model: model, 
//     sentence: "human walks and then runs and later sits down", 
//     sentence: "human waves right hand five times", 
    sentence: "human plays the guitar", 
//     sentence: "human runs runs runs walks walks walks runs runs runs", 
    prefix: "epoch_\(epoch)", 
    saveMotion: true, 
//     memoryMultiplier: 1.5,
    memoryMultiplier: 1.0,
//     memoryMultiplier: 0.65,
    motionsURL: motionsURL
)